### Simulation Generator

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
import numpy as np
from scipy.optimize import fsolve
from scipy.integrate import quad
import scipy.integrate as integrate
import numpy as np
import random
import tqdm
import pickle
# for plot and stimulate
def simulate_squid_game_v2(L, N, p, T):
    """
    Simulate the Squid Game scenario for T turns.
    
    Parameters:
    - L (int): Max number of players that can be sent back to the start line
    - N (int): Total number of players
    - p (list): List of N probabilities of failing to freeze for each player
    - T (int): Number of turns to simulate
    
    Returns:
    dict: A dictionary where the keys are player indices and the values are dictionaries.
          The inner dictionaries have keys as positions and values as the probability of 
          each player being in that position after T turns.
    """
    # Initialize players' positions
    positions = np.zeros(N, dtype=int)
    
    # Store the history of each player's position
    history = np.zeros((N, T+1), dtype=int)
    
    # Simulate the game for T turns
    for t in range(T):
        # Move all players one step forward
        positions += 1
        
        # Determine which players failed to freeze
        failed_to_freeze = np.random.rand(N) < p
        
        # If more than L players failed to freeze, find the L players who traveled the farthest
        if np.sum(failed_to_freeze) > L:
            # Find the indices of the L players who traveled the farthest and failed to freeze
            farthest_failed_indices = np.argsort(positions * failed_to_freeze)[-L:]
            
            # Send them back to the starting line
            positions[farthest_failed_indices] = 0
        else:
            # Send all players who failed to freeze back to the starting line
            positions[failed_to_freeze] = 0
        
        # Update history
        history[:, t+1] = positions
        
    # Calculate the probability distributions for each player
    prob_distributions = {}
    for i in range(N):
        # Count occurrences of each position and divide by T to get probabilities
        unique, counts = np.unique(history[i, :], return_counts=True)
        prob_distributions[i] = dict(zip(unique, counts))
    
    return prob_distributions

def plot_simulation_vs_estimate(p, result, probabilities_estimated, people_order=0,T=1000000):
    """
    Plot the simulated and estimated probabilities along with their relative error.
    
    Parameters:
    - result (dict): Output from the simulate_squid_game_v2 function
    - p (list): List of N probabilities of failing to freeze for each player
    
    Returns:
    None
    """
    # Extract position probabilities for player 0 from simulation
    #probabilities_estimated*=T
    positions_simulated = list(result[people_order].keys())
    probabilities_simulated = list(result[people_order].values())
    probabilities_simulated = np.array(probabilities_simulated)/sum(probabilities_simulated)
    

    # Estimate probabilities for the same positions
    #probabilities_estimated = [estimate(p[people_order], k) for k in positions_simulated]

    # Calculate relative errors
    rel_errors = [abs(est - sim) / sim for est, sim in zip(probabilities_estimated, probabilities_simulated)]

    # Create plots
    plt.figure(figsize=(15, 5))

    # Plot probabilities
    plt.subplot(1, 2, 1)
    plt.plot(positions_simulated, probabilities_simulated, label='Simulated')
    plt.plot(positions_simulated, probabilities_estimated, label='Estimated', linestyle='--')
    plt.xlabel(f"Position (k), p={p[people_order]}")
    plt.ylabel('Probability')
    plt.title('Probabilities: Simulated vs. Estimated')
    plt.legend()

    # Plot relative errors
    plt.subplot(1, 2, 2)
    plt.plot(positions_simulated, rel_errors, label='Rel. Error', color='red')
    plt.xlabel(f"Position (k), p={p[people_order]}")
    plt.ylabel('Relative Error')
    plt.title('Relative Error: Simulated vs. Estimated')
    plt.legend()

    # Display plots
    plt.tight_layout()
    plt.show()


### Functions: Random Setting for Simulation and Generation

In [ ]:
def generate_random_data(N):
    # 生成随机数 1, 2, 3
    random_choice = random.randint(1, 4)

    if random_choice == 1:
        # 生成 N 个在 0-1 之间的随机数
        return np.random.uniform(0, 1, N)

    elif random_choice == 2:
        # 生成两个区间
        interval1 = (random.uniform(0, 1), random.uniform(0, 1))
        interval2 = (random.uniform(0, 1), random.uniform(0, 1))

        # 从两个区间中生成数据
        half_n = N // 2
        data = np.concatenate([
            np.random.uniform(min(interval1), max(interval1), half_n),
            np.random.uniform(min(interval2), max(interval2), N - half_n)
        ])
        return data

    elif random_choice == 3:
        # 生成三个区间
        interval1 = (random.uniform(0, 1), random.uniform(0, 1))
        interval2 = (random.uniform(0, 1), random.uniform(0, 1))
        interval3 = (random.uniform(0, 1), random.uniform(0, 1))

        # 从三个区间中生成数据
        third_n = N // 3
        remaining = N - 2 * third_n
        data = np.concatenate([
            np.random.uniform(min(interval1), max(interval1), third_n),
            np.random.uniform(min(interval2), max(interval2), third_n),
            np.random.uniform(min(interval3), max(interval3), remaining)
        ])
        return data
    elif random_choice == 4:
        number_to_generate = random.randint(1, 10)
        average=N//number_to_generate
        rest=N-average*(number_to_generate-1)
        

        data_num=[average]*(number_to_generate-1)+[rest]
        
        data=[]
        for i in range(len(data_num)):
            data+=[np.random.uniform(0,1)]*data_num[i]
        return np.array(data)

# using dictionary to save initial inputs
def generate_data(times):
    dic={}
    for time in tqdm.tqdm(range(times)):
        N=random.randint(20, 200)
        L=random.choices([int(N/100+1),int(N/50+1),int(N/20+1),int(N/10),int(N/5),int(N/3),int(N/2),int(N/1)])[0]
        p=generate_random_data(N)
        T=100000
        result=simulate_squid_game_v2(L, N, p, T)
        dic[(N,L,tuple(p))]=result
    return dic

### Generation for Training Setting and Saving

In [ ]:
# 1280 pairs of data
dic_data=generate_data(1280)
with open('data_file.pkl', 'wb') as file:
    pickle.dump(dic_data, file)

### Generation for Prediction Bases on Simulation

In [ ]:
with open('data_file.pkl', 'rb') as file:
    dic_data = pickle.load(file)

In [ ]:
def solve_for_b(y1, y2, p):
    """
    根据新的等式解出逻辑斯蒂方程中的b参数。
    使用 (1-p)**b = 2 * y1 / y2 - 1 来解出 b。
    """
    return np.log(2 * y1 / y2 - 1) / np.log(1 - p)

def logistic_params(x_list, y_list, p):
    """
    根据特定条件解出逻辑斯蒂方程的参数a, b, c。
    """
    # 寻找y值最接近0.5*y_list[0]对应的x_middle
    half_y0 = 0.5 * y_list[0]
    x_middle = x_list[np.abs(y_list - half_y0).argmin()]

    # 情况1
    if y_list[1] / y_list[0] > 0.99:
        a = y_list[0]
        c = x_middle
        # 确保x_middle + 1不超出范围
        if x_middle + 1 < len(x_list):
            b = solve_for_b(y_list[x_middle], y_list[x_middle + 1], p)
        else:
            # 如果x_middle是最后一个元素，则使用x_middle和x_middle - 1
            b = solve_for_b(y_list[x_middle - 1], y_list[x_middle], p)
        #print("case 1")
    else:
        # 情况2: 找出y_list[i] - y_list[i + 1]差值最大的i，使用绝对差值
        diff = np.abs(np.diff(y_list))
        i_max_diff = np.argmax(diff)
        a = 2 * y_list[i_max_diff]
        c = x_list[i_max_diff]
        b = solve_for_b(y_list[i_max_diff], y_list[i_max_diff + 1], p)
        #print("case 2")

    return a, b, c
def generate_predict(dic_data):
    dic_predict={}
    for key in tqdm.tqdm(dic_data):
        N,L,p=key
        dic_predict[key]={}
        result=dic_data[key]
        for people_order in range(len(result)):
            x_data = np.array(list(result[people_order].keys()))
            y_data = np.array(list(result[people_order].values()))
            y_data=y_data/sum(y_data)
            personal_p=p[people_order]
            a, b, c = logistic_params(x_data, y_data, personal_p)
            dic_predict[key][people_order]=(a,b,c)
    return dic_predict
            

### Saving Prediction

In [ ]:
dic_predict=generate_predict(dic_data)
with open('predict_data_file.pkl', 'wb') as file:
    pickle.dump(dic_predict, file)